## 什么是 QLoRA 微调？

在传统的微调中，权重 ($\mathbf{W}$) 按如下方式更新：

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

其中 $L$ 是当前步骤的损失，$\eta$ 是学习率。

[LoRA](https://arxiv.org/abs/2106.09685) 尝试通过将 $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ 因式分解为两个更小的矩阵 $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ 和 $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ 来近似 $\Delta \mathbf{W}$，其中 $r \ll \text{min}(\text{d}, \text{k})$。

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

在训练过程中，只有 $\mathbf{A}$ 和 $\mathbf{B}$ 被更新，而原始权重保持冻结状态，这意味着在训练过程中只需更新一小部分（例如 <1%）原始权重。这样，在训练时可以显著减少 GPU 内存的使用，同时实现与常规（完整）微调相当的性能。

[QLoRA](https://arxiv.org/abs/2305.14314) 通过对 LLM 进行量化进一步提高了效率。例如，具有 80 亿参数的模型在 32-bit 下单独占用 32GB 的显存，而量化到 8-bit/4-bit 的 80 亿参数模型仅需分别占用 8GB/4GB 的显存。请注意，QLoRA 仅对 LLM 的权重进行低精度量化（例如 8-bit），而前向/反向计算在更高精度（例如 16-bit）下进行，LoRA 适配器的权重也保持更高精度。

### 注意

在 Kaggle 的内核上运行完整训练所需的时间过长，建议使用外部计算资源来进行完整训练。此笔记本仅使用 100 个样本用于演示。

## 安装库


1. **`"transformers>=4.42.3"`：**
   - `transformers`：这是Hugging Face的库，主要用于加载和操作预训练的Transformer模型（例如BERT、GPT、T5等）。它提供了各种用于NLP任务的模型和工具。
   - gemma-2 可在 transformers>=4.42.3 中使用

2. **`bitsandbytes`：**
   - `bitsandbytes`是一个高效的库，用于在GPU上进行低精度（如8-bit）训练和推理。它允许通过更少的内存占用来加速大规模模型的训练和推理，特别是对于大型Transformer模型（例如GPT类模型）。
   - 安装此库的目的通常是为了在有限的GPU资源下运行大模型，并且仍然保持较高的性能。

3. **`accelerate`：**
   - `accelerate`是Hugging Face提供的一个库，用于简化多GPU、多TPU分布式训练。它帮助用户轻松配置和运行复杂的并行化训练，特别是在需要跨多个设备进行计算时。
   - 安装这个库的目的是为了提高训练效率，尤其是当处理大规模数据或大模型时，它能显著减少训练时间。

4. **`peft`：**
   - `peft`（Parameter-Efficient Fine-Tuning）是用于参数高效微调的库。它提供了轻量化微调大型语言模型的方法，如LoRA（Low-Rank Adaptation），使得在有限资源的情况下也能对大型模型进行微调。
   - 安装这个库通常用于在对大模型进行定制训练时，减少微调所需的资源消耗。

这些库共同作用，使得用户能够在有限的资源下，高效地训练、微调和部署大型Transformer模型。

In [1]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


## 导入包

In [2]:
import os
import copy # 提供深拷贝功能，用于在创建对象副本时避免对原对象的修改。
from dataclasses import dataclass # 用于简化类的定义，尤其是在需要存储数据的场景。

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig, # 这是用来配置低精度计算的（如8-bit训练）。
    Gemma2ForSequenceClassification, # Gemma2的预训练模型，用于文本分类任务。
    GemmaTokenizerFast, # 快速分词器，支持高效地处理文本输入，将文本转换为模型可以理解的token序列。
    Gemma2Config, # 配置类，用于设置Gemma2模型的架构和超参数（如层数、隐藏层维度等）。
    PreTrainedTokenizerBase, # Hugging Face中所有预训练分词器的基类，定义了分词器的基本操作。
    EvalPrediction, # 在评估模型时使用的类，包含预测值和标签，用于计算评估指标。
    Trainer, # Hugging Face用于简化模型训练和评估的高阶API，集成了训练、评估、预测等功能。
    TrainingArguments, # 用于配置训练过程的参数（如学习率、批量大小、训练epoch等）。
    DataCollatorWithPadding, # 一个数据整理器，用于自动为批处理的数据填充padding，以使每个batch中的样本长度一致，方便进行并行计算。
)
'''
LoraConfig：用于配置LoRA（Low-Rank Adaptation），一种参数高效微调方法。它通过减少要调整的参数数量，允许在资源有限的情况下微调大型模型。
get_peft_model：根据LoRA配置，获取微调后的模型。
prepare_model_for_kbit_training：准备模型以进行低精度训练，例如8-bit或4-bit训练，进一步节省显存和计算资源。
TaskType：用于指定任务类型（如文本分类、生成等）。
'''
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

## 配置

### 定义一个Config类

方便存储与微调Gemma模型相关的超参数。

In [3]:
@dataclass
class Config:
    output_dir: str = "output"  # 模型输出目录
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 指定预训练模型的检查点路径。这里使用Gemma模型的4-bit量化版本
    max_length: int = 1024  # 输入序列的最大长度
    n_splits: int = 5  # 交叉验证的折数
    fold_idx: int = 0  # 当前的折索引，第0折，即第1个子集作为验证集，剩余4个子集作为训练集。
    optim_type: str = "adamw_8bit"  # 使用8-bit的AdamW优化器，节省显存
    per_device_train_batch_size: int = 2  # 每个设备的训练批量大小
    gradient_accumulation_steps: int = 2  # 梯度累积步数，表示梯度会在2个步骤后累积更新。这实际上把全局的批量大小增加到 2 * 2 = 4，有助于在小显存环境下进行更大批量的训练。
    per_device_eval_batch_size: int = 8  # 每个设备上的评估批量大小设置为8，用于验证模型的性能。在验证过程中，每个设备同时处理8个样本。
    n_epochs: int = 1  # 训练的总epoch数量
    freeze_layers: int = 16  # 冻结前16层，只微调后面的层
    lr: float = 2e-4  # 学习率
    warmup_steps: int = 20  # 预热步骤数，在训练的前20步中，学习率会从低到高逐渐增加，以防止训练开始时模型参数更新过快。
    lora_r: int = 16  # LoRA的秩，值越小，训练时的参数量越少，占用显存越少，计算量也越小
    lora_alpha: float = lora_r * 2  # LoRA的缩放因子，用来控制学习到的低秩适配器（adapter）的权重更新幅度。它通常是r的2倍
    lora_dropout: float = 0.05  # LoRA的dropout率
    lora_bias: str = "none"  # 表示LoRA不会修改偏置项，通常只对投影矩阵进行调整。
    
config = Config()

### 配置训练参数

配置语言模型训练的细节，包括输出设置、训练轮数、批次大小、梯度累积、优化器类型、混合精度等。所有这些参数为后续使用Hugging Face的Trainer类进行训练提供了基本配置。

In [4]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True, # 允许覆盖现有的输出目录。如果已经有内容存在，它们将会被新的训练结果覆盖。这对于反复实验或重新训练同一模型时很有用。
    report_to="none", # 禁用日志报告功能。这意味着不会将训练过程中的日志发送到外部工具或平台（如TensorBoard、WandB等）。
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10, # 每10个训练步打印一次日志。它控制训练过程中输出日志的频率，便于观察训练过程中的一些关键指标（如损失值）。
    eval_strategy="epoch", # 设置评估策略为epoch，即在每个epoch结束后对验证集进行评估来检查模型的表现。
    save_strategy="steps", # 模型保存的策略。设置为steps，表示按步数间隔进行保存，而不是按epoch保存。
    save_steps=200, # 每200步保存一次模型
    optim=config.optim_type,
    fp16=True, # 启用16-bit浮点数精度训练（即混合精度训练）。这有助于减少模型在训练时的显存占用，并加速计算，同时保持数值稳定性。
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

### 配置Lora

通过LoraConfig配置了LoRA的参数和应用范围，主要针对模型中的自注意力投影层，并选择性地对模型的深层进行微调（冻结前16层）。这种配置能够有效减少训练开销，使得在资源受限的环境下，也可以进行大模型的微调。

In [5]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    
    # 这是LoRA微调的目标模块。在Transformer模型中，自注意力机制的核心操作包括查询（`q_proj`）、键（`k_proj`）、和值（`v_proj`）的投影，这些模块在计算注意力分数时至关重要。
    # 通过指定只在这些模块应用LoRA，减少了要调整的参数数量，同时仍能有效影响模型的表现。
    target_modules=["q_proj", "k_proj", "v_proj"],
    
    # 决定哪些层将应用LoRA调整,将LoRA应用于从第16层到第42层的部分，忽略前16层。这种做法减少了计算开销，同时保留对深层特征的调整能力。
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias, 
    
    # 表示这是一项序列分类任务（sequence classification）。LoRA可以适应不同任务类型，这里通过指定任务类型，确保LoRA配置与任务相匹配。
    task_type=TaskType.SEQ_CLS,
)

## 初始化分词器和模型

In [6]:
# 加载预训练分词器
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint) 

# 每个输入文本的末尾将自动添加一个<eos>（End of Sequence，序列结束符）标记。
# "<eos>"标记用于告诉模型该序列已经结束，在处理生成任务或分类任务时尤其重要，因为它明确了文本的结束，避免模型无穷生成。
tokenizer.add_eos_token = True  

# 在NLP任务中，模型输入通常需要统一的长度，如果某些句子较短，需要在句子中添加填充符号（<pad>）来补齐。
# 填充方式对模型的训练和推理有一定影响，有时不同任务中使用不同的填充策略（右侧填充或左侧填充），根据模型的需求和训练方法进行调整。
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3, # 指定分类任务中有3个标签。这告诉模型我们正在处理一个三类分类问题（如情感分类中有三种情感：正面、中性、负面）。
    torch_dtype=torch.float16, # 指定模型使用16-bit浮点数（FP16）进行计算，通常用于混合精度训练，以减少显存占用和加速计算。
    device_map="auto", # 自动将模型分配到可用的硬件设备（如GPU或CPU）。如果有多个GPU，模型的不同部分可能会被分配到不同的设备，以优化资源利用。
)

# 禁用缓存机制
# 在模型推理或生成时，通常会启用缓存机制以加速推理。
# 然而，在某些情况下（如进行梯度反向传播训练时），使用缓存可能会导致显存不够，因此这里将缓存禁用。
# 对于分类任务，禁用缓存不会影响模型的训练性能，但可以避免显存溢出。
model.config.use_cache = False

# 这是PEFT（Parameter-Efficient Fine-Tuning）库中的一个函数，用于将模型准备为低比特训练模式（如8-bit或4-bit训练）。
# 大幅度减少内存使用，特别适用于大模型的微调
model = prepare_model_for_kbit_training(model)

# 使用LoRA对模型进行参数高效微调（PEFT）
model = get_peft_model(model, lora_config)
model

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-15): 16 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
              (rotary_emb): Gemma2RotaryEmbedding()
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584,

In [8]:
model.print_trainable_parameters()

trainable params: 7,891,456 || all params: 9,249,608,192 || trainable%: 0.0853


## 数据预处理

In [9]:
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
ds = ds.select(torch.arange(100))  # 只使用前100个数据进行演示

Generating train split: 0 examples [00:00, ? examples/s]

- 这个类用于从批量数据中提取提示和模型生成的响应，并将它们格式化、标记化。
- 它为两个模型的生成结果（`response_a` 和 `response_b`）进行对比，并根据哪个模型表现更好来生成分类标签。
- 最终，生成的token和标签可以用于训练或评估NLP模型，特别是分类任务（如模型选择或对话生成质量评估）。

In [10]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int # 这是标记化过程中允许的最大序列长度，超过此长度的序列将被截断。
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        '''
        处理和标记化文本
        每个输入样本有三个字段：
        prompt：表示输入的提示，前缀为"<prompt>:"。
        response_a 和 response_b：分别表示两个模型生成的响应，前缀为"<response_a>:"和"<response_b>:"。
        '''
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        # 将 prompt, response_a 和 response_b 拼接成一个字符串
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # 使用预训练的分词器将处理后的文本进行标记化，生成模型所需的输入形式。标记化结果会被截断到 max_length。
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        
        '''
        生成标签
        根据 winner_model_a 和 winner_model_b 的值，生成对应的分类标签。
        0：winner_model_a 赢。
        1：winner_model_b 赢。
        2：没有明确的赢家。
        这些标签将作为模型的训练目标（如分类模型中的类别标签）。
        '''
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
            
        # 最终返回标记化后的文本（tokenized）以及相应的标签（labels）。这些返回值将作为模型的输入和训练时的目标。
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

In [11]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)

# 对整个数据集 ds 中的所有样本进行批量处理，使用 encode 这个 CustomTokenizer 实例对每个样本进行处理。
# 处理后的数据集将包含新的字段，通常包括分词后的 input_ids、attention_mask 以及 labels。
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## 计算模型评估指标

该函数用于 Hugging Face `Trainer` 类的 `compute_metrics` 回调中。当评估模型时，`Trainer` 会自动调用这个函数，计算准确率和对数损失，并将这些指标用于模型评估或展示。

In [12]:
# 输入参数 eval_preds 是 Hugging Face transformers 库中的 EvalPrediction 对象，它包含了模型在评估集上的预测结果和真实标签。
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    
    # 从评估预测中获取模型的预测 logits 和真实标签。
    preds = eval_preds.predictions # 这是模型的预测输出，通常是 logits（未经过 softmax 的输出）。这些 logits 代表了模型对每个类别的信心分数。
    labels = eval_preds.label_ids # 这是模型的真实标签，即 label_ids。它是评估集中的实际分类标签。
    
    # 通过 softmax 将 logits 转换为概率分布
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    
    # 计算对数损失，对数损失是一个用于分类任务的损失函数，衡量预测概率与真实标签之间的差异，预测的概率越接近真实标签，损失越小。
    loss = log_loss(y_true=labels, y_pred=probs)
    
    # 通过 argmax 计算预测的类别，并基于此计算准确率。准确率是指模型预测正确的样本比例。
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    
    return {"acc": acc, "log_loss": loss}

## 交叉验证划分数据集

这段代码通过交叉验证的方式，将数据集 `ds` 划分为训练集和验证集的不同折（folds）。它使用了 `config.n_splits`（交叉验证的折数）将数据集按照折数进行分块，并创建了一个包含训练集和验证集索引的列表。

### 举例：
假设 `len(ds) = 10` 且 `config.n_splits = 5`，那么数据集有 10 个样本，进行 5 折交叉验证。结果生成的 `folds` 可能类似如下：

```python
folds = [
    (
        [1, 2, 3, 4, 6, 7, 8, 9],  # 训练集索引，排除余数为0的样本（如样本0和样本5）
        [0, 5]  # 验证集索引
    ),
    (
        [0, 2, 3, 4, 5, 7, 8, 9],  # 训练集索引，排除余数为1的样本
        [1, 6]  # 验证集索引
    ),
    # 后续折...
]
```

In [13]:
# folds 是一个列表，包含 n_splits 个这样的元组，每个元组都对应一个交叉验证折的训练集和验证集。
folds = [
    # 每个折是一个元组，元组的第一个元素是训练集的索引，第二个元素是验证集的索引。
    (
        # 这部分生成的是训练集的索引，表示在当前折中那些没有被分配到验证集的样本。
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        # 这部分生成的是验证集的索引，表示当前折所使用的验证样本。
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

## 创建Trainer，开始训练

In [14]:
# 选择数据：通过交叉验证的 folds，为当前 fold_idx 获取训练集和验证集的索引，并从数据集中选择对应的样本。
train_idx, eval_idx = folds[config.fold_idx] 

# Trainer 是 Hugging Face transformers 库中的一个高级 API，用于简化模型训练和评估过程
trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    # 对数据进行自动填充（padding），使得批次内的每个样本长度一致，以便并行计算。
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

# 训练过程：调用 train() 方法后，模型开始训练。
# 训练过程中，Trainer 会自动计算梯度、更新模型权重、记录日志、并根据配置的频率进行评估。
# 模型在每个训练周期结束时使用验证集评估性能，通过传入的 compute_metrics 函数计算诸如准确率、对数损失等指标。
trainer.train() 

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Epoch,Training Loss,Validation Loss,Acc,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,1.406200,1.834323,0.250000,1.834494,24.693300,0.810000,0.121000


TrainOutput(global_step=20, training_loss=1.4669925689697265, metrics={'train_runtime': 303.4009, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.066, 'total_flos': 2853279087925248.0, 'train_loss': 1.4669925689697265, 'epoch': 1.0})